In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
# hyperparameters
lr = 0.001
training_iters = 100000
batch_size = 128 

n_inputs = 28 # 每一次input一行pixel,即n_inputs = 28
n_steps = 28 # 一共28列,因此n_steps = 28
n_hidden_units = 128 # neurons in hidden layer
n_classes = 10

In [6]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])

In [7]:
# Define weights and biases
weights = {
    # (28, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # (128, 10)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units,])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes,]))
}

In [15]:
def RNN(X, weights, biases):
    # hidden layer for inut to cell
    # X(128 batch, 28 steps, 28 inputs) ==> X(128 * 28, 28 inputs)
    # 将展成一个长度为28的向量,一共128 * 28个(reshape中第一个参数为-1,表示个数由总数除以shape得到)
    X = tf.reshape(X, [-1, n_inputs])
    # X_in : (128 batch * 28 steps, 128 hidden)
    X_in = tf.matmul(X, weights['in']) + biases['in']
    # X_in ==> (128 batch, 28 steps, 128 hidden)
    X_in = tf.reshape(X_in, [-1, n_steps, n_hidden_units])
    
    # LSTM cell
    cell = tf.contrib.rnn.BasicLSTMCell(n_hidden_units)
    init_state = cell.zero_state(batch_size, dtype=tf.float32)
    # dynamic RNN比传统RNN具有更好的效果
    outputs, final_state = tf.nn.dynamic_rnn(cell, X_in, initial_state=init_state, time_major=False)
    
    outputs = tf.unstack(tf.transpose(outputs, [1, 0, 2]))
    # hidden layer for output as the final results
    results = tf.matmul(states[1], weights['out']) + biases['out']
    
    return results

In [16]:
pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_login(logits=pred, labels=y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

# tf.argmax此函数对矩阵按行或列计算最大值，0表示列，1表示行
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# tf.cast()将矩阵元素转换成需要的类型
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "I:\Anaconda\lib\site-packages\tensorflow\python\framework\ops.py", line 1269, in __init__
    self._traceback = _extract_stack()
  File "I:\Anaconda\lib\site-packages\tensorflow\python\framework\ops.py", line 2506, in create_op
    original_op=self._default_original_op, op_def=op_def)
  File "I:\Anaconda\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 767, in apply_op
    op_def=op_def)
